### Boltzman - Bostom Home Prices (Keras)

Redoing one of the earlier problems using Keras, just to get the hang of it.


In [46]:
from keras.datasets import boston_housing
from keras.optimizers import SGD
import numpy as np

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [47]:
# we try to get out inputs and outputs as close to variance 1 as possible.  This makes it easier for gradient descent to converge

y_stddev = y_train.std()
y_mean = y_train.mean()

def normalize_data(x):
    x_mean = x.mean(axis=0)
    x_stddev = x.std(axis=0)
    return (x - x_mean) / x_stddev 

x_train = normalize_data(x_train) 
y_train = normalize_data(y_train)
x_test = normalize_data(x_test)
y_test = normalize_data(y_test)


In [48]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import logging

model = Sequential([
    Dense(1, input_shape=(13,)), # We want 1 layer.  with one output.  We have 13 weigths
    Activation('linear'),  # activation function
])

optimizer = SGD(lr = 0.01) # stochastic gradient descent with learning rate.
model.compile(loss='mse', optimizer=optimizer) # model will minimize mean squared error.  

model.fit(x_train, y_train, batch_size=16, epochs=100) # training...
score = model.evaluate(x_test, y_test, batch_size=16) # evauluate on test data.

# logging.info('mse=%f,' % (score))
print(score)

Epoch 1/100
404/404 [==============================] - 0s 385us/step - loss: 1.5247
Epoch 2/100
404/404 [==============================] - 0s 77us/step - loss: 0.5567
Epoch 3/100
404/404 [==============================] - 0s 84us/step - loss: 0.4251
Epoch 4/100
404/404 [==============================] - 0s 78us/step - loss: 0.3637
Epoch 5/100
404/404 [==============================] - 0s 80us/step - loss: 0.3303
Epoch 6/100
404/404 [==============================] - 0s 77us/step - loss: 0.3100
Epoch 7/100
404/404 [==============================] - 0s 90us/step - loss: 0.2969
Epoch 8/100
404/404 [==============================] - 0s 86us/step - loss: 0.2882
Epoch 9/100
404/404 [==============================] - 0s 78us/step - loss: 0.2834
Epoch 10/100
404/404 [==============================] - 0s 79us/step - loss: 0.2797
Epoch 11/100
404/404 [==============================] - 0s 91us/step - loss: 0.2754
Epoch 12/100
404/404 [==============================] - 0s 77us/step - loss: 0.2750


In [49]:
def calc_errors(y_train, y_predictions):
    absolute_errors = np.abs(y_train - y_predictions) * y_stddev
    mean_absolute_error = np.mean(absolute_errors)

    squared_errors = ((y_train - y_predictions) * y_stddev) ** 2
    mean_squared_error = np.mean(squared_errors)

    mean_absolute_percent_error = np.mean(
        np.abs(absolute_errors / y_mean)
    )

    return (mean_absolute_error, mean_squared_error, mean_absolute_percent_error)

(mean_absolute_error, mean_squared_error, mean_absolute_percent_error) = calc_errors(
    y_train,
    y_train.mean()
)
print(
    f"Baseline Mean Abs Err: {mean_absolute_error:0.1f} | "
    f"Baseline Mean Squared Err: {mean_squared_error:0.2f} | "
    f"Baseline Mean Abs %Err: {mean_absolute_percent_error:0.2f}"
)

(mean_absolute_error, mean_squared_error, mean_absolute_percent_error) = calc_errors(
    y_train,
    # model.predict gives us a (404, 1) matrix which won't play well
    # with our (404,) shape y_train. Thus we reshape the output to
    # (404,).
    model.predict(x_train).reshape((-1))
)
print(
    f"Model Mean Abs Err: {mean_absolute_error:0.1f} | "
    f"Model Mean Squared Err: {mean_squared_error:0.2f} | "
    f"Model Mean Abs %Err: {mean_absolute_percent_error:0.2f}"
)

Baseline Mean Abs Err: 6.6 | Baseline Mean Squared Err: 84.62 | Baseline Mean Abs %Err: 0.30
Model Mean Abs Err: 3.4 | Model Mean Squared Err: 22.13 | Model Mean Abs %Err: 0.15
